# Optimizing the parameters and hyperparameters of our LSTM model

In [41]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.metrics import mean_squared_error
import random

# Preprocessing the data

In [42]:
# Load the data
data = pd.read_csv('EURUSD_data')

# Select features and target
features = data[['Open', 'High', 'Low', 'Close', 'Adj Close']]
target = data['Close']

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(features)

# Create sequences of data for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length, 3])  # Use the 'Close' price as target
    return np.array(X), np.array(y)

seq_length = 60  # Using 60 days of data to predict the next day's price
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Defining our LSTM with variable parameters

In [43]:
# Function to create and compile LSTM model
def create_lstm_model(units, dropout_rate, learning_rate):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units//2, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    return model

# Defining function to randomly select paramters

In [44]:
# Random Search for hyperparameter tuning
def random_search(X_train, y_train, X_test, y_test, param_dist, n_iter=10):
    best_model = None
    best_mse = float('inf')
    best_params = None
    
    for i in range(n_iter):
        # Randomly select hyperparameters
        units = random.choice(param_dist['units'])
        dropout_rate = random.choice(param_dist['dropout_rate'])
        learning_rate = random.choice(param_dist['learning_rate'])
        batch_size = random.choice(param_dist['batch_size'])
        epochs = random.choice(param_dist['epochs'])
        
        print(f"Model {i+1}: units={units}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")
        
        # Create and train the model
        model = create_lstm_model(units, dropout_rate, learning_rate)
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=0)
        
        # Evaluate the model
        predictions = model.predict(X_test)
        mse = mean_squared_error(y_test, predictions)
        print(f"Model {i+1} MSE: {mse}")
        
        # Check if this model is the best so far
        if mse < best_mse:
            best_mse = mse
            best_model = model
            best_params = {'units': units, 'dropout_rate': dropout_rate, 'learning_rate': learning_rate, 'batch_size': batch_size, 'epochs': epochs}
    
    return best_model, best_params, best_mse

# Defining the domain of parameters

In [45]:
# Define the parameter distributions
param_dist = {
    'units': [75, 100, 125],
    'dropout_rate': [0.1, 0.2, 0.3],
    'learning_rate': [0.001, 0.005, 0.01],
    'batch_size': [16, 32, 64],
    'epochs': [80]
}

# Performing the search

In [46]:
# Perform Random Search
best_model, best_params, best_mse = random_search(X_train, y_train, X_test, y_test, param_dist, n_iter=20)

print("Best Parameters:", best_params)
print("Best MSE:", best_mse)

Model 1: units=100, dropout_rate=0.2, learning_rate=0.005, batch_size=32, epochs=80
16/16 [==============================] - 1s 9ms/step
Model 1 MSE: 0.0002650981446509463
Model 2: units=100, dropout_rate=0.1, learning_rate=0.005, batch_size=16, epochs=80
16/16 [==============================] - 1s 9ms/step
Model 2 MSE: 8.484297766234736e-05
Model 3: units=100, dropout_rate=0.1, learning_rate=0.005, batch_size=32, epochs=80
16/16 [==============================] - 1s 9ms/step
Model 3 MSE: 0.0003616536749957337
Model 4: units=100, dropout_rate=0.3, learning_rate=0.005, batch_size=32, epochs=80
16/16 [==============================] - 1s 8ms/step
Model 4 MSE: 0.00013983103512066827
Model 5: units=75, dropout_rate=0.2, learning_rate=0.01, batch_size=64, epochs=80
16/16 [==============================] - 1s 6ms/step
Model 5 MSE: 0.0001315812050638234
Model 6: units=125, dropout_rate=0.1, learning_rate=0.01, batch_size=16, epochs=80
16/16 [==============================] - 1s 13ms/step
Mode

In [ ]:
# Save the best model
best_model.save('best_lstm_model.h5')